In [1]:
import pandas as pd
import numpy as np
from scipy.stats import truncnorm
import re
import random
import geopandas as gpd
from numba import jit, njit
from shapely import MultiPoint, Point
from shapely.ops import nearest_points

READ STREEN NETWORK NODES FOR POSITION SNAPPING

In [ ]:
streetNetwork = gpd.read_file('c:/Users/Administrator/Documents/pt2network/streetNetwork.gpkg')
streetNetwork_nodes = gpd.read_file('c:/Users/Administrator/Documents/pt2network/streetNetwork_nodes.gpkg')
roadPoints = streetNetwork_nodes['geometry']
roadNodes = MultiPoint(roadPoints)

<span style="font-size:5em;">CREATE PERSONS</span>

In [2]:
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)


In [3]:
profileData = pd.read_csv("/Users/lukeguardino/Documents/SILO_crap/MontrealZones/ProfileDataByZone3.csv")

/var/folders/7p/74395j5j7jg7r8chb_tyb6s80000gn/T/ipykernel_66796/1360461993.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251

In [4]:
zoneSystem = pd.read_csv("/Users/lukeguardino/Documents/SILO_crap/MontrealZones/SkimsTravelTimes/zoneSystem.csv", usecols=['Zone', 'Area'])

In [5]:
profileData = profileData[(profileData['Population']!='-') & (profileData['Geography'].isin(zoneSystem['Zone']))]
totalPersons = profileData['Population'].astype(int).sum()

In [6]:
salary_distribution_employed = get_truncated_normal(mean=70000, sd=40000, low=30000, upp=200000)
salary_distribution_retired = get_truncated_normal(mean=40000, sd=20000, low=15000, upp=70000)

In [7]:
employment_incomes = salary_distribution_employed.rvs(int(totalPersons*.9))
retired_incomes = salary_distribution_retired.rvs(int(totalPersons*.1))

Nationalities

In [ ]:
# taken from wikipedia general nationality distribution
tmp = """Canadian 11,135,965
English	6,320,085
Scottish	4,799,010
French	4,670,595
Irish	4,627,000
German	3,322,405
Chinese	1,769,195
Italian	1,587,970
First Nations	1,525,565
Indian	1,374,710
Ukrainian	1,359,655
Dutch	1,111,655
Polish	1,106,585
Filipino 837,130"""

countries = tmp.replace('\t', ' ').replace(',', '').split('\n')
nationalities = {}
total_nationalities_pop = 0
for i in countries:
    ind = re.search('[0-9]', i).span()[0]
    nationalities[i[:ind-1]] = 0.0
    total_nationalities_pop += int(i[ind:])

for i in countries:
    ind = re.search('[0-9]', i).span()[0]
    nationalities[i[:ind-1]] = int(i[ind:])/total_nationalities_pop

print(nationalities)
nationalities_list = random.choices(list(nationalities.keys()), weights=list(nationalities.values()),k=persons.shape[0])

Synthesize persons with initial data

In [ ]:
persons = pd.DataFrame(columns=['id', 'hhId', 'age', 'gender', 'relationship', 'occupation', 'occupation_type', 'workplace', 'income', 'schoolplace', 'nationality'])
person_nats = persons['nationality']
p_cols = persons.columns[:-1]
p = lambda x: p_cols.get_loc(x)
persons:np.ndarray = persons[p_cols].values

# convert relationship to number value
for i in range(len(persons)):
    rel = persons[i][4]
    res = -1
    if rel=='SINGLE': 
        res=0
    elif rel=='MARRIED': res=1
    elif rel=='CHILD': res=2
    persons[i][4] = res
persons = persons.astype(np.int32)

zone population counts in numpy format for numba

In [8]:
personZoneCounts = np.empty([0,2])
for i, zone in zoneSystem.iterrows():
    row = profileData.loc[profileData['Geography']==zone['Zone']]
    population = 0
    try:
        # if there is a population number for this zone
        zonePopulation = row['Population'].astype(int).iloc[0]
    except:
        print('No population for zone: ', zone['Zone'])
    
    personZoneCounts = np.r_[personZoneCounts, [[zone['Zone'], zonePopulation]]]

No population for zone:  24640126.0
No population for zone:  24650247.0
No population for zone:  24650374.0
No population for zone:  24660044.0
No population for zone:  24661260.0
No population for zone:  24661280.0
No population for zone:  24661282.0
No population for zone:  24661285.0
No population for zone:  24661694.0
No population for zone:  24661729.0
No population for zone:  24661778.0
No population for zone:  24661801.0
No population for zone:  24661872.0
No population for zone:  24662067.0
No population for zone:  24662115.0
No population for zone:  24662247.0
No population for zone:  24662658.0
No population for zone:  24662942.0
No population for zone:  24663142.0
No population for zone:  24663271.0
No population for zone:  24663282.0
No population for zone:  24663329.0
No population for zone:  24663334.0
No population for zone:  24663335.0
No population for zone:  24663339.0
No population for zone:  24663340.0
No population for zone:  24663351.0
No population for zone:  246

create persons as numpy array so it's faster

In [9]:
@njit
def createPersons(personCounts):
    p = np.empty((0,10))
    for zoneNum in range(personCounts.shape[0]):
        numPeopleInZone = personCounts[zoneNum][1]
        for person in range(numPeopleInZone):
            age = np.random.randint(0,100)
            gender = np.random.randint(0,2)
            # 0:SINGLE 1:MARRIED 2:CHILD
            relationship = 2 if age < 18 else np.random.randint(0, 2)
            occupation = 0 if age < 4 \
                    else 4 if age > 65 \
                    else 3 if age < 22 \
                    else 1 if np.random.random() < .93 \
                    else 2
            occupation_type = 1
            workplace = -1
            income=-1
            if occupation==1:
                income = employment_incomes[np.random.randint(0,int(totalPersons*.9))]
            elif occupation==4:
                income = retired_incomes[np.random.randint(0,int(totalPersons*.1))]
            
            schoolplace=-1
            
            p = np.append(p, np.array([[personZoneCounts[zoneNum][0], -1, age, gender, relationship, occupation, occupation_type, workplace, income, schoolplace]]), axis=0)
        print(zoneNum)
    return p
    

create dataframe from np array of persons

In [ ]:
persons = pd.DataFrame(data=createPersons(personZoneCounts), columns=['id', 'hhId', 'age', 'gender', 'relationship', 'occupation', 'occupation_type', 'workplace', 'income', 'schoolplace'])

add nationalities to persons df

In [ ]:
persons['nationality'] = nats

Old method to create persons

In [ ]:



personCount = 0
for i, zone in zoneSystem['Zone'].iterrows():
    row = profileData.loc[profileData['Geography']==zone]
    population = 0
    try:
        # if there is a population number for this zone
        zonePopulation = row['Population'].astype(int).iloc[0]
    except:
        print('No population for zone: ', zone)
    for numPerson in range(zonePopulation): # run for each person in the current zone
        persons.loc[personCount, 'id'] = personCount # set the id of the current person. ID ranges from 0..totalPersons
        persons.loc[personCount, 'hhId'] = -1 #set household to -1 for now, change later when we marry couples (c code so is faster)
        age = random.randint(0, 100)
        persons.loc[personCount, 'age'] = age
        
        persons.loc[personCount, 'gender'] = random.randint(0, 1)
        persons.loc[personCount, 'relationship'] = 'CHILD' if age < 18 else ['SINGLE','MARRIED'][random.randint(0, 1)]
        #0:toddler 1:employed 2:unemployed 3:Student 4:Retiree
        employment_rate = -1
        try:
            employment_rate = row['Employment rate'].astype(float).iloc[i] # get employment rate for zone
        except:
            pass
        occupation = \
                    0 if age < 4 \
                    else 4 if age > 65 \
                    else 3 if age < 25 \
                    else 1 if random.random() < employment_rate \
                    else 2
        persons.loc[personCount, 'occupation'] = occupation
        persons.loc[personCount, 'occupation_type'] = 1 #full-time employment
        persons.loc[personCount, 'workplace'] = -1 #id of job, to be set later
        persons.loc[personCount, 'income'] = salary_distribution_employed.rvs(1) if occupation==1 else salary_distribution_retired.rvs(1) if occupation==4 else 0.0
        persons.loc[personCount, 'schoolplace'] = random.randint(0, 100) if occupation==3 else -1 # id of school if attending
        persons.loc[personCount, 'nationality'] = nationalities_list[personCount]
        personCount+=1


    print(personCount,'/',totalPersons, 'in zone:', zone)


In [ ]:
persons.to_csv('./pp_2006_.csv', index=False)

<h1>CREATE DWELLINGS</h1>

In [ ]:
dwelling_counts = pd.read_csv('/Users/lukeguardino/Documents/SILO_crap/MontrealZones/DwellingTypesByZone2006.csv')

In [ ]:
dwellings = pd.DataFrame(columns=['id', 'zone', 'type', 'hhId', 'bedrooms', 'quality', 'monthlyCost', 'restriction', 'yearBuilt', 'floor', 'building', 'usage', 'coordX', 'coordY'])

In [ ]:
dwelling_counts = dwelling_counts[dwelling_counts['Geography'].isin(zoneSystem['Zone'])]
dwelling_counts = dwelling_counts.reset_index(drop=True)

In [ ]:
polygons = []
for i in range(zoneShape.shape[0]):
    polygons.append(zoneShape.iloc[i]['geometry'])
    
geoseries = gpd.GeoSeries(polygons, dwelling_counts['Geography'])
dwelling_coords = geoseries.sample_points(size=dwelling_counts['Total - Structural type of dwelling'].astype(float).astype(int))
dwelling_coords = dwelling_coords.explode(index_parts=True)

In [ ]:
total_dwelling_count_GMA = float(dwelling_counts['Total - Structural type of dwelling'].astype(float).sum())
print(total_dwelling_count_GMA)

In [ ]:
dwelling_number = 0
for i,dwelling in dwelling_counts.iterrows():
    numDwellings = int(float(dwelling['Total - Structural type of dwelling']))
    starttime = time.time()
    for j in range(numDwellings):
        point = Point(float(dwelling_coords[dwelling['Geography']][j].x), float(dwelling_coords[dwelling['Geography']][j].y))
        p1, p2 = nearest_points(point, roadNodes)

        dwellings.loc[dwelling_number] = [dwelling_number, int(dwelling['Geography']), ['SFD', 'SFA', 'MF234', 'MF5plus', 'MH'][random.randint(0,4)], -1, -1, -1, -1, -1, -1, -1, -1, -1, p2.x, p2.y]
        dwelling_number+=1
    print(dwelling_number,'/',total_dwelling_count_GMA, '--- zone', dwelling['Geography'])


<h1>CREATE JOBS</h1>

CLEAN DATA

In [ ]:
tmpJobData = pd.read_csv('./jobData.csv', encoding_errors='replace')
for i in range(tmpJobData.shape[0]):
    geog = str(tmpJobData.loc[i, 'Geography']).split(' ')[0]
    try:
        float(geog)
        tmpJobData.loc[i, 'Geography'] = geog
    except:
        print('drop', geog)
        

tmpJobData.to_csv('./jobData_clean.csv', index=False)

NEW DATAFRAME FOR JOBS

In [ ]:
jobs = pd.DataFrame(columns=['id','zone','personId','type','coordX','coordY','startTime','duration'])

READ CLEANED JOB DATA

In [ ]:
zoneShape = gpd.read_file('/Users/lukeguardino/Documents/SILO_crap/MontrealZones/GreaterMontrealZones2006_2138.shp')

In [ ]:
CT_job_data = pd.read_csv('./jobData_clean.csv', encoding_errors='replace', dtype=str)
CT_job_data = CT_job_data[(CT_job_data['Total - Industry - Sectors - North American Industry Classification System (NAICS) 2017']!='..') & (CT_job_data['Total - Industry - Sectors - North American Industry Classification System (NAICS) 2017']!='x')]

In [ ]:
print("Total Jobs in area:", float(CT_job_data['Total - Industry - Sectors - North American Industry Classification System (NAICS) 2017'].astype(int).sum()))

np.int64(3166035)

EXPAND CT AREA JOB DATA FOR EACH DA

In [ ]:
jobs_per_da = pd.DataFrame(columns=['DAUID', 'CTUID', 'numJobs'])
for i, ct_area in CT_job_data.iterrows():
    ct_area_jobs = int(ct_area['Total - Industry - Sectors - North American Industry Classification System (NAICS) 2017'])
    da_areas = zoneShape.loc[zoneShape['CTUID']==ct_area['Geography'], 'DAUID']
    for da_in_ct in da_areas:
        da_jobs = int(ct_area_jobs/da_areas.shape[0])
        jobs_per_da.loc[len(jobs_per_da)] = [da_in_ct, ct_area['Geography'], da_jobs]

In [ ]:
polygons = []
for i in range(zoneShape.shape[0]):
    polygons.append(zoneShape.iloc[i]['geometry'])
print(len(polygons))

geoseries = gpd.GeoSeries(polygons, zoneShape['DAUID'])
job_coords = geoseries.sample_points(size=jobs_per_da.sort_values(['DAUID'], ascending=True)['numJobs'].astype(int))
job_coords = job_coords.explode(index_parts=True)

4999


In [ ]:
job_headers = ["11 Agriculture, forestry, fishing and hunting","21 Mining, quarrying, and oil and gas extraction",'22 Utilities','23 Construction','31-33 Manufacturing','41 Wholesale trade','44-45 Retail trade','48-49 Transportation and warehousing','51 Information and cultural industries','52 Finance and insurance','53 Real estate and rental and leasing',"54 Professional, scientific and technical services",'55 Management of companies and enterprises',"56 Administrative and support, waste management and remediation services",'61 Educational services','62 Health care and social assistance',"71 Arts, entertainment and recreation",'72 Accommodation and food services','81 Other services (except public administration)','91 Public administration']

CONVERT JobData.csv HEADER TYPES TO SILO TYPES

In [ ]:
def translate_job_type(job_typ):
    typ = ''
    match job_typ:
        case "11 Agriculture, forestry, fishing and hunting":
            typ = 'Agri'
        case "21 Mining, quarrying, and oil and gas extraction":
            typ = 'Mnft'
        case "22 Utilities":
            typ = 'Util'
        case "23 Construction":
            typ = 'Cons'
        case "31-33 Manufacturing":
            typ = 'Mnft'
        case "41 Wholesale trade":
            typ = 'Serv'
        case "44-45 Retail trade":
            typ = 'Retl'
        case "48-49 Transportation and warehousing":
            typ = 'Trns'
        case "51 Information and cultural industries":
            typ = 'Serv'
        case "52 Finance and insurance":
            typ = 'Finc'
        case "53 Real estate and rental and leasing":
            typ = 'Rlst'
        case "54 Professional, scientific and technical services":
            typ = 'Serv'
        case "55 Management of companies and enterprises":
            typ = 'Admn'
        case "56 Administrative and support, waste management and remediation services":
            typ = 'Admn'
        case "61 Educational services":
            typ = 'Serv'
        case "62 Health care and social assistance":
            typ = 'Serv'
        case "71 Arts, entertainment and recreation":
            typ = 'Serv'
        case "72 Accommodation and food services":
            typ = 'Serv'
        case "81 Other services (except public administration)":
            typ = 'Serv'
        case "91 Public administration":
            typ = 'Admn'
        case _:
            pass
    return typ


SYNTHESIZE JOBS FOR EACH ZONE INCLUDED IN JobData.csv

In [ ]:
jobs = pd.DataFrame(columns=['id','zone','personId','type','coordX','coordY','startTime','duration'])
numJobs = 0
total_jobs = CT_job_data['Total - Industry - Sectors - North American Industry Classification System (NAICS) 2017'].astype(float).sum()
for i, ct_area in CT_job_data.iterrows():
    ct_area_jobs = int(ct_area['Total - Industry - Sectors - North American Industry Classification System (NAICS) 2017'])
    da_areas = zoneShape.loc[zoneShape['CTUID']==ct_area['Geography'], 'DAUID']
    job_types = ct_area[job_headers]
    try:
        job_dist = random.choices(list(job_types.index), list(job_types.values.astype(int)), k=ct_area_jobs)
    except:
        print('no jobs in zone')
    for da_in_ct in da_areas:
        da_jobs = int(ct_area_jobs/len(da_areas))
        for job_num in range(da_jobs):
            # snap position to nearest road 
            point = Point(float(job_coords[da_in_ct][job_num].x), float(job_coords[da_in_ct][job_num].y))
            
            p1, p2 = nearest_points(point, roadNodes)
            jobs.loc[len(jobs)] = [numJobs, da_in_ct, -1, translate_job_type(random.choice(job_dist)), p2.x, p2.y, 32400, 28800]
            numJobs+=1
    print(str(numJobs/total_jobs*100)+'%')

jobs.to_csv('./jj_2006.csv', index=False)

<span style="font-size:5em;">DATA LINKING</span>

INITIAL DATA READ

In [ ]:
dwellings_file = 'testOutput/dd_2006.csv'
jobs_file = './jj_2006_2138_snapped.csv'
persons_file = 'testOutput/pp_2006.csv'

In [ ]:
dwellings = pd.read_csv(dwellings_file)
d_cols = dwellings.columns
d = lambda x: d_cols.get_loc(x)
dwellings:np.ndarray = dwellings.values

jobs = pd.read_csv(jobs_file, usecols=['id','zone','personId','coordX','coordY','startTime','duration'], dtype=np.float32)
job_types = pd.read_csv(jobs_file, usecols=['type'])
j_cols = jobs.columns
j = lambda x: j_cols.get_loc(x)
jobs:np.ndarray = jobs.values

persons = pd.read_csv(persons_file)
nats = persons['nationality']
p_cols = persons.columns[:-1]
p = lambda x: p_cols.get_loc(x)
persons:np.ndarray = persons[p_cols].values
for i in range(len(persons)):
    rel = persons[i][4]
    res = -1
    if rel=='SINGLE': 
        res=0
    elif rel=='MARRIED': res=1
    elif rel=='CHILD': res=2
    persons[i][4] = res
persons = persons.astype(np.float32)

households = pd.DataFrame(columns=['id', 'dwelling', 'hhSize', 'autos'], dtype=int)
h_cols = households.columns
h = lambda x: h_cols.get_loc(x)
households:np.ndarray = households.values


NUMBA FUNCTIONS

In [ ]:
@jit(nopython=True)
def findPartner(n, person):
    for i in range(n, persons.shape[0]):
        if persons[i][4]==1 and persons[i][1]==-1 and \
                                        abs(person[2] - persons[i][2]) < 10:
            # print(person[0], 'married to ', persons[i][0])
            return persons[i]
    return None

In [ ]:
@jit(nopython=True)
def firstLonelyChild(n):
    for i in range(n, persons.shape[0]):
        if persons[i][1]==-1 and persons[i][4]==2:
            lastIndex = i
            return int(persons[i][0])

In [ ]:
@jit(nopython=True)
def nextUnhousedPerson(n):
    for i in range(n, persons.shape[0]):
        if persons[i][1]==-1:
            return int(persons[i][0])

In [ ]:
@jit(nopython=True)
def getNextJob(n):
    for i in range(n, jobs.shape[0]):
        if jobs[i][2] == -1:
            return jobs[i][0]

HOUSEHOLD SYNTHESIS

for each dwelling
- create a household
  - has either a single person and 0-1 kids
  - OR married couple with 0-2 kids
- 

In [ ]:
personNum = 0
lastChildIndex = 0
err = 1
for i, dwelling in enumerate(dwellings): # for each dwelling
    hhId = i
    if persons[personNum][1]==-1: # if current person is unhoused
        person = persons[personNum] # random.randint(0, len(persons)-1)
        personNum += 1
    else:
        next = nextUnhousedPerson(personNum) # skip to the index of the next unhoused person to save time
        if next == None:
            err=2
            break
        person = persons[next]
        personNum = int(person[0])
    persons[personNum][p('hhId')] = hhId
    hhSize = 1
    numKids=0
    if person[p('relationship')]==1: # married
        partner = findPartner(int(personNum), person)
        if partner == None: # can't find partner for person
            persons[personNum][p('relationship')]=0 # set person to single
            numKids = random.randint(0,1)
            hhSize += numKids
        else:  
            persons[int(partner[0])][1] = hhId
            numKids = random.randint(0,3)
            hhSize += numKids+1

        for j in range(numKids):
            child_ind = firstLonelyChild(lastChildIndex)
            if child_ind != None:
                persons[child_ind][1] = hhId
                lastChildIndex = child_ind
        

    elif person[p('relationship')]==0: # single
        numKids=random.randint(0,1)
        if numKids>0:
            child_ind = firstLonelyChild(lastChildIndex)
            if child_ind != None:
                persons[child_ind][1] = hhId
                lastChildIndex = child_ind
        hhSize += numKids

    households = np.r_[households, [[hhId, dwelling[d('id')], hhSize, random.randint(0,hhSize-numKids)]]]
    dwellings[i][d('hhId')] = hhId

    if i%10000==0:
        print(i)
    if i%150000==0:
        num=int(i/150000)
        print('writing to file:', num)
        pd.DataFrame(data=households, columns=h_cols).astype(int).to_csv('./testOutput/hh_2006_'+str(num)+'.csv', index=False)
        households = np.empty([0,4])

if err==1:
    print("Exhausted Dwellings")
elif err==2:
    print("Exhausted Persons")


In [ ]:
lastJobIndex = 0
for i in range(len(persons)):
    job = getNextJob(lastJobIndex)
    print('person', i, 'job', job)
    if job != None:
        persons[i][7] = job
        jobs[int(job)][2] = persons[i][0]
        lastJobIndex = int(job)
    else:
        print('no more jobs')
        break

    if i%10000==0:
        print('person: ', i)

In [ ]:
outputPersons = pd.DataFrame(data=persons, columns=p_cols)
outputPersons['nationality'] = nats
outputPersons['relationship'] = outputPersons['relationship'].apply(lambda r: 'SINGLE' if r==0 else 'MARRIED' if r==1 else 'CHILD' if r==2 else -1)

<h1>CREATE DEVELOPMENT FILE</h1>

READ DWELLING COUNTS FROM FILE

<small>Read and filtered above in dwelling synthesis</small>

In [ ]:
development = pd.DataFrame(columns=['Zone','SFD','SFA','MF234','MF5plus','MH','DevCapacity','DevLandUse'])

for i, row in zoneSystem.iterrows():
    cap = 200
    landUse = 0.31
    if str(int(row['Zone'])) in dwelling_counts['Geography'].astype(str).values:
        print('in')
        cap = int(float(dwelling_counts.loc[dwelling_counts['Geography'].astype(str) == str(int(row['Zone'])), 'Total - Structural type of dwelling'].iloc[0]))
        landUse = float(row['Area'])
    development.loc[len(development)] = [int(row['Zone']), 1, 1, 1, 1, 1, cap*1.2, landUse*2.47]

In [39]:
development = development.astype({'Zone': 'int64', 'SFD': 'int64', 'SFA': 'int64', 'MF234': 'int64', 'MF5plus': 'int64', 'MH': 'int64', 'DevCapacity': 'int64'})

,Zone,SFD,SFA,MF234,MF5plus,MH,DevCapacity,DevLandUse
0,24550077.0,1.0,1.0,1.0,1.0,1.0,216.0,0.872404
1,24550078.0,1.0,1.0,1.0,1.0,1.0,210.0,0.270465
2,24550079.0,1.0,1.0,1.0,1.0,1.0,372.0,0.716794
3,24550080.0,1.0,1.0,1.0,1.0,1.0,354.0,0.889447
4,24550081.0,1.0,1.0,1.0,1.0,1.0,330.0,13.994032


In [40]:
development.to_csv('./development.csv', index=False)